# Crystal graph convolution

In [ ]:
import numpy as np
import pickle
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, GELU, Dropout

In [ ]:
from torch_geometric.nn import CGConv, global_add_pool, global_mean_pool
from torch_geometric.data import Data, DataLoader

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
from torchinfo import summary

## Load data

In [ ]:
DATA_FOLDER = '../data'
PICKLE_FOLDER = '../pickles'

In [ ]:
with open(f'{PICKLE_FOLDER}/timeseries.pickle', 'rb') as f:
    ts = pickle.load(f)

In [ ]:
total_samples, total_brain_regions, ts_length = ts.shape

In [ ]:
df_metadata = pd.read_csv(f'{DATA_FOLDER}/patients-cleaned.csv', index_col=0)

In [ ]:
df_metadata.head(2)

### Select connectivity dataset

In [ ]:
THRESHOLD = 0.1                                          # 0.01, 0.05, 0.1, 0.15
N = 20 #                                                 # 3, 5, 7, 10, 15, 20, 40
CORR_TYPE = 'pearson'                                    # 'pearson', 'spearman', 'partial-pearson'
THRESHOLD_METHOD = 'abs-group-avg-diff'                  # 'abs-sample-diff', 'abs-group-avg-diff'
THRESHOLD_TYPE = 'min'                                 # 'min', 'max' or for kNN 'small', 'large'
KNN = False                                               # Whether all or only top N neigbors are taken

In [ ]:
fc_folder = f'{PICKLE_FOLDER}/fc-{CORR_TYPE}{"-knn" if KNN else ""}-{THRESHOLD_METHOD}'

# Try Gini or SGD.
# fc_folder = f'{PICKLE_FOLDER}/fc-{CORR_TYPE}-gini'
# fc_folder = f'{PICKLE_FOLDER}/fc-{CORR_TYPE}-sgd'

In [ ]:
fc_file_binary = f'{fc_folder}/{THRESHOLD_TYPE}-{f"knn-{N}" if KNN else f"th-{THRESHOLD}"}-binary.pickle'
fc_file_real = f'{fc_folder}/{THRESHOLD_TYPE}-{f"knn-{N}" if KNN else f"th-{THRESHOLD}"}-real.pickle'

# fc_file_binary = f'{fc_folder}/binary.pickle'
# fc_file_real = f'{fc_folder}/real.pickle'

In [ ]:
with open(fc_file_binary, 'rb') as f:
    edge_index_matrix = pickle.load(f)

In [ ]:
with open(fc_file_real, 'rb') as f:
    fc_matrix = pickle.load(f)

In [ ]:
edge_index_matrix.shape

In [ ]:
fc_matrix.shape

## Split data

In [ ]:
with open(f'{PICKLE_FOLDER}/test-indices.pickle', 'rb') as f:
    test_indices = pickle.load(f)

In [ ]:
train_indices = list(set(range(total_samples)) - set(test_indices))

In [ ]:
train_targets = df_metadata.iloc[train_indices]["target"].reset_index(drop=True)

In [ ]:
print(f'Train set size: {len(train_indices)}')
print(f'Test set size: {len(test_indices)}')

## Extend dataset

In [ ]:
N_SURROGATES = 5                    # 5
SURROGATE_METHOD = 'iaaft'          # 'iaaft', 'aaft'

In [ ]:
with open(f'{PICKLE_FOLDER}/timeseries-{SURROGATE_METHOD}-{N_SURROGATES}.pickle', 'rb') as f:
    ts_surrogates = pickle.load(f)

In [ ]:
print(f'Extra data: {ts_surrogates.shape}')
print(f'{ts_surrogates.shape[0] / total_samples} surrogates per sample')

Extend only training data. Test set will consist of true data only.

In [ ]:
ts_train_surrogates = np.concatenate([ts_surrogates[N_SURROGATES*i:N_SURROGATES*i+N_SURROGATES] for i in train_indices], axis=0)

In [ ]:
ts_train_surrogates.shape

## Prepare data

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### `Data` object fields

- `data.x`: Node feature matrix with shape `[num_nodes, num_node_features]`

- `data.edge_index`: Graph connectivity in COO format with shape `[2, num_edges]` and type `torch.long`

- `data.edge_attr`: Edge feature matrix with shape `[num_edges, num_edge_features]`

- `data.y`: Target to train against (may have arbitrary shape), e.g., node-level targets of shape `[num_nodes, *]` or graph-level targets of shape `[1, *]`

- `data.pos`: Node position matrix with shape `[num_nodes, num_dimensions]`

### Select node features

- onehot
- timeseries
- correlations

In [ ]:
ts_length_start = 50
ts_length_end = ts_length

# A part of signal serves as node features.
def timeseries_in_nodes(i):
    return torch.from_numpy(ts[i][:,ts_length_start:ts_length_end]).to(torch.float32)

In [ ]:
# Each nodes contains its row from FC matrix.
def correlations_in_nodes(i):
    return torch.from_numpy(fc_matrix[i]).to(torch.float32)

In [ ]:
def corr_and_ts_in_nodes(i):
    corr_i = torch.from_numpy(fc_matrix[i]).to(torch.float32)
    ts_i = torch.from_numpy(ts[i][:,ts_length_start:ts_length_end]).to(torch.float32)
    return torch.cat([corr_i, ts_i], axis=1)

In [ ]:
# Each brain region is onehot encoded. See GIN for phenotype paper.
def onehot_in_nodes(i):
    return torch.diag(torch.ones(total_brain_regions))

In [ ]:
features_in_nodes = correlations_in_nodes
# total_brain_regions / ts_length_end-ts_length_start
num_features_in_nodes = total_brain_regions

### Create dataset

In [ ]:
dataset = [Data(
    x=features_in_nodes(i),  
    edge_index=torch.from_numpy(np.asarray(np.nonzero(edge_index_matrix[i]))).to(torch.int64),
    edge_attr=torch.from_numpy(fc_matrix[i][fc_matrix[i] != 0]).reshape((-1,1)).to(torch.float32),
    y=torch.tensor([target], dtype=torch.int64)
).to(device) for target, i in zip(train_targets, train_indices)]

In [ ]:
print(f'True train data: {len(dataset)}')

In [ ]:
# Extension.
dataset_ext = [Data(
    x=features_in_nodes(i),  
    edge_index=torch.from_numpy(np.asarray(np.nonzero(edge_index_matrix[i]))).to(torch.int64),
    edge_attr=torch.from_numpy(fc_matrix[i][fc_matrix[i] != 0]).repeat_interleave(num_features_in_nodes).reshape(-1,num_features_in_nodes).to(torch.float32),
    y=torch.tensor([target], dtype=torch.int64)
).to(device) for target, i, ts_surr in zip(
    np.repeat(train_targets, N_SURROGATES),
    np.repeat(train_indices, N_SURROGATES),
    ts_train_surrogates
)]

In [ ]:
print(f'Surrogate train data: {len(dataset_ext)}')

### Join surrogates with true data

In [ ]:
# dataset.extend(dataset_ext)

# train_targets_ext = np.concatenate([train_targets, np.repeat(train_targets, N_SURROGATES)])
# train_targets_ext.shape

In [ ]:
print(f'Extended train data: {len(dataset)}')

In [ ]:
print('Data object')
print(f'Edge index: {dataset[0].edge_index.shape}')
print(f'Edge features: {dataset[0].edge_attr.shape}')
print(f'Node features: {dataset[0].x.shape}')
print(f'Target: {dataset[0].y.shape}')

## Define model

In [ ]:
from typing import Union, Tuple
from torch_geometric.typing import PairTensor, Adj, OptTensor, Size

import torch
from torch import Tensor
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d
from torch_geometric.nn.conv import MessagePassing


class CGConvFixed(MessagePassing):

    def __init__(self, channels: Union[int, Tuple[int, int]], dim: int = 0,
                 aggr: str = 'add', batch_norm: bool = False,
                 bias: bool = True, **kwargs):
        super(CGConvFixed, self).__init__(aggr=aggr, **kwargs)
        self.channels = channels
        self.dim = dim
        self.batch_norm = batch_norm

        if isinstance(channels, int):
            channels = (channels, channels)

        self.lin_f = Linear(2*channels[0] + dim, channels[1], bias=bias)
        self.lin_s = Linear(2*channels[0] + dim, channels[1], bias=bias)
        self.bn = BatchNorm1d(channels[1])

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_f.reset_parameters()
        self.lin_s.reset_parameters()
        self.bn.reset_parameters()


    def forward(self, x: Union[Tensor, PairTensor], edge_index: Adj,
                edge_attr: OptTensor = None, size: Size = None) -> Tensor:
        if isinstance(x, Tensor):
            x: PairTensor = (x, x)

        # propagate_type: (x: PairTensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=size)
        out = self.bn(out) if self.batch_norm else out
        #out += x[1]
        return out


    def message(self, x_i, x_j, edge_attr: OptTensor) -> Tensor:
        if edge_attr is None:
            z = torch.cat([x_i, x_j], dim=-1)
        else:
            z = torch.cat([x_i, x_j, edge_attr], dim=-1)
        return self.lin_f(z).sigmoid() * F.softplus(self.lin_s(z))

    def __repr__(self):
        return '{}({}, dim={})'.format(self.__class__.__name__, self.channels,
                                       self.dim)

In [ ]:
class CGNet(torch.nn.Module):
    
    def __init__(self, hidden_channels, eps=0., p_dropout=0.5):
        super(CGNet, self).__init__()
        
        self.conv1 = CGConv((num_features_in_nodes, hidden_channels), dim=0)
        self.conv2 = CGConv((hidden_channels, hidden_channels), dim=0)
        self.conv3 = CGConv((hidden_channels, hidden_channels), dim=0)

        self.fc1 = Linear(hidden_channels*3, 2)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        # 1. Obtain node embeddings
        # 2. Remember layer outputs.
        x = self.conv1(x, edge_index)
        out1 = global_mean_pool(x, batch)
        x = self.conv2(x, edge_index)
        out2 = global_mean_pool(x, batch)
        x = self.conv3(x, edge_index)
        out3 = global_mean_pool(x, batch)

        # 3. Concatenate itermediate outputs.
        x = torch.cat((out1, out2, out3), 1)

        # 4. Fully connected for final classification.
        x = self.fc1(x)     # CELoss already incorporates `log_softmax`.
        
        return x

In [ ]:
# Architecture.
CGNet(hidden_channels=8)

## Evaluation

In [ ]:
def evaluation_metrics(predicted, labels):
    pred_positives = predicted == 1
    label_positives = labels == 1

    tp = (pred_positives & label_positives).sum().item()
    tn = (~pred_positives & ~label_positives).sum().item()
    fp = (pred_positives & ~label_positives).sum().item()
    fn = (~pred_positives & label_positives).sum().item()

    return tp, tn, fp, fn

## Train model

In [ ]:
NUM_FOLDS = 3

In [ ]:
skf = StratifiedKFold(n_splits=NUM_FOLDS, random_state=42, shuffle=True)

In [ ]:
# Settings.
EPOCHS = 500
LR = 0.001
MOMENTUM = 0.8
OPTIMIZER = 'sgd'
LOSS = 'ce'
BATCH_SIZE = 8

VALIDATE_FREQ = 10

HIDDEN_CHANNELS = 90

STEP_SIZE = 50
GAMMA = 0.5

WEIGHT_DECAY = 0.0001

settings_str = f'bs={BATCH_SIZE},e={EPOCHS},lr={LR},mom={MOMENTUM},opt={OPTIMIZER},loss={LOSS},step={STEP_SIZE},gamma={GAMMA},wd={WEIGHT_DECAY}'

In [ ]:
# Experiment folder.
EXP_FOLDER = 'runs/cg'

In [ ]:
# Experiment.
EXP_ID = 6

In [ ]:
for kfold, (train_index, val_index) in enumerate(skf.split(np.zeros(len(train_targets)), train_targets)):

    # Init TB writer.
    experiment_str = f'id={EXP_ID:03d},fold={kfold},{settings_str}'
    writer = SummaryWriter(f"../{EXP_FOLDER}/{experiment_str}")

    # Init model.
    net = CGNet(hidden_channels=HIDDEN_CHANNELS).to(device)
    # optimizr = torch.optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    optimizr = torch.optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    criterion = torch.nn.CrossEntropyLoss()
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizr, step_size=STEP_SIZE, gamma=GAMMA)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizr, lr_lambda)

    # Save architecture.
    with open(f"../{EXP_FOLDER}/{experiment_str}/architecture", 'w', encoding="utf-8") as f:
        f.write(fc_folder + '\n')
        f.write(fc_file_binary + '\n')
        f.write(fc_file_real + '\n')
        f.write(features_in_nodes.__str__() + '\n')
        f.write('\n'.join(experiment_str.split(',')) + '\n\n')
        f.write(net.__str__() + '\n\n')
        f.write(str(summary(net)))

    # Prepare data.
    X_train = [dataset[i] for i in train_index]
    X_val = [dataset[i] for i in val_index]
    
    trainloader = DataLoader(X_train, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(X_val, batch_size=BATCH_SIZE, shuffle=False)

    # Train.
    for epoch in range(EPOCHS):
        running_loss = 0.
        
        net.train()
        for data in trainloader:
            
            optimizr.zero_grad()
            outputs = net(data)   
            loss = criterion(outputs, data.y)  # Compute the loss.
            loss.backward()  # Derive gradients.
            optimizr.step()
            running_loss += loss.item()

        #scheduler.step()    # Update LR.
        epoch_loss = running_loss / len(trainloader)
        writer.add_scalar('training loss', epoch_loss, epoch)

        running_loss = 0.

        # Evaluate epoch.
        tp, tn, fp, fn = 0, 0, 0, 0
        total = 0

        net.eval()
        for data in valloader:

            optimizr.zero_grad()
            outputs = net(data) 
            loss = criterion(outputs, data.y)  # Compute the loss.
            running_loss += loss.item()

            if (epoch+1) % VALIDATE_FREQ == 0:
                predicted = outputs.argmax(dim=1)
                labels = data.y.view(-1)

                # Update.
                tp_, tn_, fp_, fn_ = evaluation_metrics(predicted, labels)
                tp += tp_; tn += tn_; fp += fp_; fn += fn_
                total += data.y.size(0)

        epoch_loss = running_loss / len(valloader)
        writer.add_scalar('validation loss', epoch_loss, epoch)
        
        if (epoch+1) % VALIDATE_FREQ == 0:
            writer.add_scalar('validation accuracy', (tp + tn) / total, epoch)
            writer.add_scalar('validation precision', tp / (tp + fp) if (tp + fp) > 0 else 0, epoch)
            writer.add_scalar('validation recall', tp / (tp + fn), epoch)
    
    # Single fold during exploration.
    #break

print('Finished training')